# Rotten tomatoes, Top 100 Movies

**link:** https://www.rottentomatoes.com/top/bestofrt/

**Rotten Tomatoes** is an American review-aggregation website for film and television. The company was launched in August 1998 by three undergraduate students at the University of California, Berkeley: Senh Duong, Patrick Y. Lee, and Stephen Wang. Although the name "Rotten Tomatoes" connects to the practice of audiences throwing rotten tomatoes when disapproving of a poor stage performance, the original inspiration comes from a scene featuring tomatoes in the Canadian film Léolo (1992).

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = requests.get('https://www.rottentomatoes.com/top/bestofrt/')

In [4]:
soup = BeautifulSoup(url.text, 'lxml')

In [5]:
# print(soup.prettify())

In [6]:
table = soup.find('table', class_='table')

In [7]:
request_movie = requests.get('https://www.rottentomatoes.com' + table.find_all('tr')[36].a['href'])
movie_soup = BeautifulSoup(request_movie.text, 'lxml')

In [8]:
tomatometre = movie_soup.find_all('span', class_='mop-ratings-wrap__percentage')[0].text.strip()
audience_rating = movie_soup.find_all('span', class_='mop-ratings-wrap__percentage')[1].text.strip()

In [9]:
for label in movie_soup.find_all('div', 'meta-label subtle'):
    if label.text == 'Directed By:':
        director_tag = movie_soup.find_all('div', 'meta-value')[movie_soup.find_all('div', 'meta-label subtle').index(label)]
        director = ''
        for a in director_tag.find_all('a'):
            director += a.text + ', '
        print(director.strip(', '))
        
    if label.text == 'Runtime:':
        runtime = movie_soup.find_all('div', 'meta-value')[movie_soup.find_all('div', 'meta-label subtle').index(label)].text
        print(runtime.strip().split(' ')[0])

Stanley Donen, Gene Kelly
102


In [10]:
import pandas as pd

RowList = []

for movie in table.find_all('tr')[1:]:
    rank = movie.td.text.strip('.')
    movie_title = movie.a.text.split('(')[0].strip()
    movie_year = movie.a.text.strip()[-5:-1]
    noofreviews = movie.find('td', class_='right hidden-xs').text 
    movie_link = 'https://www.rottentomatoes.com' + movie.a['href']
    
    request_movie = requests.get('https://www.rottentomatoes.com' + movie.a['href'])
    movie_soup = BeautifulSoup(request_movie.text, 'lxml')
    
    tomatometre = movie_soup.find_all('span', class_='mop-ratings-wrap__percentage')[0].text.strip()
    audience_rating = movie_soup.find_all('span', class_='mop-ratings-wrap__percentage')[1].text.strip()
    
    director = ''
    runtime = ''
    
    for label in movie_soup.find_all('div', 'meta-label subtle'):
        if label.text == 'Directed By:':
            director_tag = movie_soup.find_all('div', 'meta-value')[movie_soup.find_all('div', 'meta-label subtle').index(label)]
            director = ''
            for a in director_tag.find_all('a'):
                director += a.text + ', '
            director = director.strip(', ')

        if label.text == 'Runtime:':
            runtime = movie_soup.find_all('div', 'meta-value')[movie_soup.find_all('div', 'meta-label subtle').index(label)].text
            runtime = runtime.strip().split(' ')[0]
    
    
    # print([rank, movie_title, movie_year, noofreviews, director, movie_link, tomatometre, audience_rating, runtime])
    RowList.append([rank, movie_title, movie_year, noofreviews, director, movie_link, tomatometre, audience_rating, runtime])

In [11]:
pd.DataFrame(RowList, columns = ['Rank', 'Movie title', 'Release year', 
                                 'No of reviews', 'Director', 'Movie link', 
                                 'Tomatometer', 'Audience ratings', 'Run time(Minutes)']).to_csv('RottenTomatoesTop100.csv', index=False)